In [1]:
%run main.py

  0%|                                                                                    | 0/169001437 [00:00<?, ?it/s]

START STATS ON OVERFITTING WITH CIFAR100 :  [100]
START CIFAR100
START TRAINING TARGET MODEL


100%|███████████████████████████████████████████████████████████████| 169001437/169001437 [00:23<00:00, 7321659.79it/s]


Extracting ../data\cifar-100-python.tar.gz to ../data
DATASET SIZE {'train': 2500, 'val': 1000}


C:\Users\isaro\anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:216: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Training complete in 2m 55s
DONE TRAIN
START TRAINING SHADOW MODEL
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in 2m 44s
DONE TRAIN
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in 2m 41s
DONE TRAIN
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in 2m 55s
DONE TRAIN
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in 2m 33s
DONE TRAIN
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in 2m 44s
DONE TRAIN
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in 2m 28s
DONE TRAIN
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in 3m 10s
DONE TRAIN
Files already downloaded and verified
DATASET SIZE {'train': 2500, 'val': 1000}
Training complete in

C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metr

C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\isaro\anaconda3\lib\site-packages\sklearn\metr

END CIFAR100


In [1]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# Example dataset: You can replace this with your own dataset loading mechanism
data = load_digits()
X, y = data.data, data.target

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Resampling using SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Define model (Random Forest, Logistic Regression, or SVM)
model = RandomForestClassifier(random_state=42)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_resampled, y_train_resampled)

# Get best model from GridSearch
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model using various metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Use weighted average to account for class imbalance
recall = recall_score(y_test, y_pred, average='weighted')  # Use weighted average for recall

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision (weighted):", precision)
print("Recall (weighted):", recall)

# Store the results in arrays
all_res_accuracy = [accuracy]
all_res_precision = [precision]
all_res_recall = [recall]

# If there are multiple iterations, you can accumulate the results in a list and save them at the end
np.save("results/res_accuracy.npy", np.array(all_res_accuracy))
np.save("results/res_precision.npy", np.array(all_res_precision))
np.save("results/res_recall.npy", np.array(all_res_recall))

# Save additional metrics like per-class precision, recall if necessary
# These can be calculated for each class and saved as needed
from sklearn.metrics import classification_report
report = classification_report(y_test, y_pred, output_dict=True)

# Saving per-class metrics in a dictionary and converting to numpy for storage
precision_per_class = np.array([report[str(i)]['precision'] for i in range(len(report) - 3)])  # Remove 'accuracy', 'macro avg', 'weighted avg'
recall_per_class = np.array([report[str(i)]['recall'] for i in range(len(report) - 3)])
accuracy_per_class = np.array([report[str(i)]['f1-score'] for i in range(len(report) - 3)])

# Saving the per-class metrics
np.save("results/res_precision_per_class.npy", precision_per_class)
np.save("results/res_recall_per_class.npy", recall_per_class)
np.save("results/res_accuracy_per_class.npy", accuracy_per_class)


Accuracy: 0.975
Precision (weighted): 0.9756983830084134
Recall (weighted): 0.975
